In [124]:

from fastapi import HTTPException
from openai import OpenAI
import os
import instructor
from dotenv import load_dotenv
from groq import Groq
from typing import List
import json
from pydantic import BaseModel
from together import Together


load_dotenv()

groq_client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)
together = Together()

class DialogueEntry(BaseModel):
    character: str
    text: str

class ComicPage(BaseModel):
    scene: str
    dialogue: List[DialogueEntry]  # List of dialogues
    image_prompt: str
    # final_transition: str
    # image_url: str

class ComicScript(BaseModel):
    summary: str
    pages: List[ComicPage]  # A list of pages
    
    # characters: List[str]

def groq_text_generation(system_prompt, prompt):
    try:
        # Generate comic script using Groq
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            # model="llama-3.3-70b-specdec",
            # model="llama-3.1-8b-instant",
            response_model=ComicScript,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            temperature=0.65,
        )
        # Extract structured response
        comic_data = response.model_dump()
        # print("============= comic_data parsed", comic_data)
        # print(json.dumps(comic_data['pages'], indent=4))
        return comic_data #['pages']
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Groq Error: {str(e)}")

In [70]:
from openai import OpenAI
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def openai_text_generation(system_prompt, prompt):
    # Generate comic script using OpenAI
    try:
        completion = openai.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}, #"A cyberpunk detective is investigating a missing android in a neon-lit city."
            ],
            response_format=ComicScript,  # Structured Pydantic validation
        )

        # Access the structured response
        comic_data = completion.choices[0].message.parsed

        comic_list = comic_data.model_dump()#['pages']
        # print("============= comic_data parsed", comic_data)

        return comic_list
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"OpenAI Error: {str(e)}")

In [ ]:
def together_text_generation(system_prompt, prompt):
    extract = together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": ComicScript.model_json_schema(),
        },
    )

    output = json.loads(extract.choices[0].message.content)
    print(json.dumps(output, indent=2))
    return output

In [125]:


# system_prompt = """
# The user will provide some exam text. Please parse the "question" and "answer" and output them in JSON format. 

# EXAMPLE INPUT: 
# Which is the highest mountain in the world? Mount Everest.

# EXAMPLE JSON OUTPUT:
# {
#     "question": "Which is the highest mountain in the world?",
#     "answer": "Mount Everest"
# }
# """
def deepseek_text_generation(system_prompt, prompt):
    client = OpenAI(
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        base_url="https://api.deepseek.com",
    )

    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
            'type': 'json_object'
        }
    )

    print(json.loads(response.choices[0].message.content))

In [8]:
system_prompt="""
You are a professional comic book creator. Generate a detailed 4-page comic script based on the user's prompt. The response must be a JSON object following this structure:\n"
"- 'pages': A list of pages.\n"
"- Each page includes:\n"
"  - 'scene': A vivid description of the visual setting.\n"
"  - 'dialogue': A list of dictionaries where each dictionary contains 'character' and 'text' fields.\n"
"Ensure a coherent storyline across pages. Content should be in the language of the story. Keep only scene in English and make it can easier for ai image generation
"""

In [78]:
system_prompt= """ 
You are an expert comic scriptwriter & visual scene designer, specializing in Action-Adventure | Kid-Friendly comics. Your job is to take the user's story idea and generate a structured multi-page comic script in JSON format, ensuring engaging storytelling, dynamic visuals, and smooth scene transitions.
Show me only 4 pages of the generated comic script. Try to make the scenes as exciting as possible and keep the story finish or moving forward.
Core Features & Rules
1. Language Handling
Detect the user's input language automatically.
If the input is in Vietnamese, return content only in Vietnamese.
2. Genre & Tone
Always maintain an Action-Adventure | Kid-Friendly tone.
Keep the language simple, engaging, and fun for young readers.
Use lighthearted rivalries, humor, and teamwork instead of dark or violent themes.
Story & Character Adaptation
3. More Engaging Storytelling
Introduce obstacles & challenges: Ensure that every story has unexpected twists, rivalries, or problems that the main character must overcome.
Build excitement: Make scenes progressively more thrilling (e.g., from "a normal race" → "unexpected obstacles" → "a dramatic final showdown").
Add emotional depth: Ensure dialogue reflects characters’ emotions, not just simple lines.
4. Stronger Character Development
If the user provides character details, incorporate them.
If not, create characters that fit the adventure theme.
Ensure each character’s dialogue is expressive and personality-driven (e.g., a confident hero, a mischievous rival, a supportive friend).
Show character growth throughout the story (e.g., from overconfident → learns teamwork → overcomes challenge).
Visual & Scene Enhancements
5. Enhanced Image Prompts (Comic Style)
All image_prompt outputs must be in English, regardless of the user's input language.
Ensure AI-generated images accurately match each scene description by including:
Dynamic perspective (e.g., low-angle action shots, cinematic framing).
Character actions & emotions (e.g., "Max grips the steering wheel, eyes locked on the track").
Environment details (e.g., "dust flying, neon-lit city background, stormy sky").
Lighting & motion effects (e.g., "bright sunlight casting long shadows, motion blur on speeding cars").
Use this base format for every image_prompt:

"comic style, highly detailed scene, dynamic perspective, immersive composition, [character actions], [environment details], [lighting effects], [motion effects], [specific scene details]"
6. More Motion & Sound Prompts
Describe how characters and environments move (e.g., "Max's monster truck jumps over a rocky hill, wheels spinning mid-air").
Include sound effects to enhance comic energy:
Engine roar: VROOOM
Sudden stop: SKRRRT
Crash or impact: BOOM
Wind or speed effects: WHOOSH
JSON Output Format (Comic Script)
Every output must follow this structured JSON format with rich storytelling, motion, and sound prompts:

json
```
{
  "title": "[Generated Comic Title Based on User's Prompt]",
  "genre": "Action-Adventure | Kid-Friendly",
  "summary": "[Short overview of the user's story idea]",
  "characters": {
    "[Character Name]": {
      "description": "[Visual description]",
      "personality": "[Short bio & personality traits]"
    }
  },
  "pages": [
    {
      "page": 1,
      "title": "[Scene Title]",
      "scene_description": "[What happens in this scene? Include challenges, emotions, or key character moments.]",
      "image_prompt": "(only in English) comic style,  [character actions], [environment details], [lighting effects], [motion effects], [specific scene details]",
      "motion_prompt": "[Describe how the characters and environment move dynamically]",
      "sound_prompt": "[Describe the background sounds & effects for immersive storytelling]",
      "dialogue": [
        {
          "character": "[Character Name]",
          "text": "[Expressive and personality-driven dialogue]"
        }
      ],
      "final_transition": "[Sentence leading to the next scene, keeping story momentum]"
    }
  ]
}
"""

In [ ]:
system_prompt= """ 
You are an expert comic scriptwriter & visual scene designer, specializing in Action-Adventure | Kid-Friendly comics. Your job is to transform the user’s story idea into a structured multi-page comic script in JSON format, ensuring compelling storytelling, dynamic visuals, and smooth transitions between scenes. The output should be immersive, engaging, and highly detailed to support AI-generated illustrations and seamless comic reading.
Show me only 4 pages of the generated comic script. Try to make the scenes as exciting as possible and keep the story finish or moving forward.

📌 Core Features & Rules
1. Language Handling
Detect the user's input language automatically.
If the input is in Vietnamese, return content only in Vietnamese, but image_prompt must always be in English to ensure optimal AI-generated images.
2. Genre & Tone
Always maintain an Action-Adventure | Kid-Friendly tone, ensuring the story is exciting and appropriate for younger readers.
Keep the language simple, engaging, and fun, using lighthearted humor, teamwork, and friendly rivalries instead of dark or violent themes.
Introduce adventure-driven tension that is thrilling but never overly serious or frightening.
📌 Story & Character Development
3. Engaging Storytelling
Each story must follow a clear beginning, middle, and end, progressing through logical and exciting developments.
Include unexpected obstacles, twists, and surprises to keep the narrative engaging.
Ensure the protagonist faces real challenges that require problem-solving and adaptability rather than making the race too easy.
The resolution should be satisfying, reflecting teamwork, skill, or an important lesson.
4. Limited & Stronger Character Arcs
The story should have a maximum of 3 characters to maintain focus and ensure deep character interactions.
If the user specifies characters, incorporate them naturally into the adventure. If not, create well-developed characters that fit the story’s theme.
Ensure each character has a distinct personality and their dialogue reflects it.
Rivalries should feel fun and competitive, allowing characters to shift between opposition and teamwork when needed.
Show character progression throughout the story (e.g., a reckless character learns to strategize, or a selfish character discovers teamwork).
📌 Visual & Scene Enhancements
5. Enhanced Image Prompts (Comic Style in English Only)
All image_prompt outputs must be in English, regardless of the user's input language.
Ensure each image_prompt includes:
Each image_prompt must accurately match the scene, reflecting unique character actions, race conditions, and challenges.
Do NOT repeat static elements like sky color or background details unless relevant to the scene.
Dynamic perspectives (e.g., low angles, wide shots, motion blur effects).
Character actions & emotions (e.g., "Max grips the steering wheel, eyes locked on the track").
Environmental details (e.g., "dust flying, neon-lit city background, stormy sky").
Lighting & motion effects (e.g., "bright sunlight casting long shadows, motion blur on speeding cars").
**Use this base format for every image_prompt, in english only:**

"comic style, highly detailed scene, dynamic perspective, immersive composition, [character actions], [environment details], [lighting effects], [motion effects], [specific scene details]"
6. Motion & Sound Effects for Better Immersion
Clearly describe how characters and objects move in each scene (e.g., "Max’s tires spin wildly as he takes a sharp turn, dust flying in his wake").
Include sound effects to increase comic-style energy, ensuring they are formatted in bold uppercase letters:
Engine roars: "VROOOM"
Sudden stops: "SKRRRT"
Explosions or impacts: "BOOM"
Wind or speed effects: "WHOOSH"
📌 Structured JSON Output Format (Comic Script)
Every output must follow this structured JSON format, ensuring consistency, clarity, and engaging storytelling.

json
{
  "title": "[Generated Comic Title Based on User's Prompt]",
  "genre": "Action-Adventure | Kid-Friendly",
  "summary": "[Short overview of the user's story idea]",
  "characters": {
    "[Character Name]": {
      "description": "[Visual description]",
      "personality": "[Short bio & personality traits]"
    }
  },
  "pages": [
    {
      "page": 1,
      "title": "[Scene Title]",
      "scene_description": "[What happens in this scene? Include challenges, emotions, or key character moments.]",
      "text_full": "[Expanded, highly descriptive storytelling for this scene]",
      "image_prompt": "comic style, highly detailed scene, dynamic perspective [specific scene detail in english related to the scene]",
      "motion_prompt": "[Describe how the characters and environment move dynamically]",
      "sound_prompt": "[Describe the background sounds & effects for immersive storytelling]",
      "dialogue": [
        {
          "character": "[Character Name]",
          "text": "[Expressive and personality-driven dialogue]"
        }
      ],
      "final_transition": "[Sentence leading to the next scene, keeping story momentum]"
    }
  ]
}
"""

In [101]:
prompt="monster truck sang Mỹ tham gia cuộc đua"
groq_text_generation(system_prompt, prompt)

{'summary': 'Cuộc đua off-road đầy phấn khích với sự tham gia của một chiếc monster truck từ Mỹ',
 'pages': [{'scene': 'Khởi động cuộc đua',
   'dialogue': [{'character': 'Tài xế Mỹ',
     'text': 'Chúng ta bắt đầu rồi! Hãy sẵn sàng để chứng kiến sức mạnh của chiếc monster truck này!'},
    {'character': 'Khán giả', 'text': 'Woo-hoo! Cuộc đua bắt đầu!'}],
   'image_prompt': 'comic style, highly detailed scene, dynamic perspective, immersive composition, monster truck speeding, dusty off-road track, bright sunlight casting long shadows, motion blur on speeding truck, cheering crowd in the background'},
  {'scene': 'Vượt chướng ngại vật',
   'dialogue': [{'character': 'Tài xế Mỹ',
     'text': 'Đến chướng ngại vật đầu tiên! Hãy xem chiếc xe này có thể làm gì!'},
    {'character': 'Khán giả', 'text': 'Wow, nó quá mạnh!'}],
   'image_prompt': 'comic style, highly detailed scene, dynamic perspective, immersive composition, monster truck jumping over obstacles, mud flying, dramatic lighting 

In [115]:
prompt="monster truck sang Mỹ tham gia cuộc đua"
openai_text_generation(system_prompt, prompt)

{'summary': 'Câu chuyện về một chiếc xe tải quái vật từ Mỹ tham gia vào một cuộc đua đầy bất ngờ và thú vị.',
 'pages': [{'scene': 'Giới thiệu Xe Tải Quái Vật',
   'dialogue': [{'character': 'Max',
     'text': 'Xem nào, giới thiệu với các bạn, tôi là Max – chiếc xe tải quái vật mạnh mẽ nhất trong đội!'}],
   'image_prompt': "comic style, highly detailed scene, dynamic perspective, Max the monster truck revving its engine, bright stadium background filled with cheering fans, sunlight glistening off the chrome, excitement in the air, engines roaring everywhere, VROOOM sounds echoing through the crowd, Max's tires kicking up dust from the racetrack as he speeds forward, SKKKRRRT sounds as he skids around a corner, showing off his skills and strength with confident poses and expressions of determination, playful competitiveness in the air with colorful banners around the stadium and vibrant excitement of the fans watching, a feeling of thrill and anticipation, bright blue skies above, dyn

In [126]:
together_text_generation(system_prompt, prompt)

InvalidRequestError: Error code: 400 - {"message": "Input validation error", "type_": "invalid_request_error"}

In [136]:
system_prompt_v1 = """
The user will provide some story idea. Please parse the "summary" and "pages" and output them in JSON format. 

Show only 4 pages of the generated comic script. Try to make the scenes as exciting as possible and keep the story finish or moving forward.

📌 Core Features & Rules
1. Language Handling
Detect the user's input language automatically.
If the input is in Vietnamese, return content only in Vietnamese, but image_prompt must always be in English to ensure optimal AI-generated images.

EXAMPLE INPUT: 
The highest mountain in the world? Mount Everest.

EXAMPLE JSON OUTPUT:
{
"summary": "[Short overview of the user's story idea]",
"pages": [
    {
      "page": 1,
      "title": "[Scene Title]",
      "scene_description": "[What happens in this scene? Include challenges, emotions, or key character moments.]",
      "text_full": "[Expanded, highly descriptive storytelling for this scene]",
      "image_prompt": "comic style, highly detailed scene, dynamic perspective [specific scene detail in english related to the scene]",
      "dialogue": [
        {
          "character": "[Character Name]",
          "text": "[Expressive and personality-driven dialogue]"
        }
      ],
      "final_transition": "[Sentence leading to the next scene, keeping story momentum]"
    }
  ]
}
"""


{'pages': [{'page': 1, 'title': 'Khởi Hành', 'scene_description': 'Một chiếc monster truck đầy màu sắc chuẩn bị rời Việt Nam để sang Mỹ. Đám đông tụ tập để chào tạm biệt, không khí đầy phấn khích và hồi hộp.', 'text_full': 'Trong một buổi sáng đẹp trời, chiếc monster truck được trang trí lộng lẫy với màu sắc rực rỡ đang chuẩn bị cho chuyến đi dài từ Việt Nam sang Mỹ. Đám đông tụ tập xung quanh, chụp ảnh và gửi lời chúc tốt đẹp. Không khí tràn ngập niềm vui và sự mong đợi.', 'image_prompt': 'comic style, highly detailed scene, a colorful monster truck surrounded by a crowd, dynamic perspective showing the truck and excited people waving goodbye, morning light', 'dialogue': [{'character': 'Người lái xe', 'text': 'Chúng ta sẽ làm nên lịch sử!'}, {'character': 'Một người trong đám đông', 'text': 'Chúc các bạn may mắn!'}], 'final_transition': 'Với tiếng động cơ gầm rú, chiếc monster truck bắt đầu hành trình dài, hướng về phía chân trời mới.'}, {'page': 2, 'title': 'Trên Đường', 'scene_descr

In [ ]:
deepseek_text_generation(system_prompt_v1, "kể chuyện về monster truck sang Mỹ.")

In [135]:
[{'page': 1, 'title': 'Khởi Đầu Cuộc Phiêu Lưu', 'scene_description': "Một chiếc monster truck đầy màu sắc, tên là 'Thunder Beast', chuẩn bị rời Việt Nam để sang Mỹ tham gia một giải đấu lớn. Nhóm kỹ sư và tài xế đang kiểm tra lần cuối trước khi lên đường.", 'text_full': 'Trong một xưởng sửa chữa nhộn nhịp tại Hà Nội, Thunder Beast, chiếc monster truck được trang bị động cơ mạnh mẽ và thiết kế độc đáo, đang được chuẩn bị cho chuyến đi định mệnh. Không khí hồi hộp và phấn khích bao trùm khi mọi người làm việc không ngừng nghỉ.', 'image_prompt': 'comic style, highly detailed scene, dynamic perspective of a colorful monster truck being prepared in a bustling workshop, engineers and driver checking the vehicle, vibrant atmosphere', 'dialogue': [{'character': 'Nam', 'text': 'Mọi thứ đã sẵn sàng chưa? Chúng ta không thể để xảy ra bất kỳ sai sót nào!'}, {'character': 'Hùng', 'text': 'Yên tâm đi, Thunder Beast sẽ khiến cả nước Mỹ phải kinh ngạc!'}], 'final_transition': 'Với mọi thứ đã sẵn sàng, Thunder Beast bắt đầu hành trình dài hướng tới nước Mỹ, nơi những thử thách lớn đang chờ đợi.'}, {'page': 2, 'title': 'Hành Trình Xuyên Đại Dương', 'scene_description': 'Thunder Beast được vận chuyển bằng tàu biển. Trên đường đi, họ gặp phải một cơn bão lớn, thử thách đầu tiên cho cả đội.', 'text_full': 'Giữa biển khơi mênh mông, Thunder Beast và đội ngũ của nó đối mặt với cơn bão dữ dội. Sóng lớn đập vào tàu, khiến mọi thứ trở nên hỗn loạn. Nhưng với sự dũng cảm và kỹ năng, họ vượt qua được thử thách.', 'image_prompt': 'comic style, highly detailed scene, dynamic perspective of a monster truck on a ship battling a fierce storm at sea, waves crashing, intense atmosphere', 'dialogue': [{'character': 'Nam', 'text': 'Giữ chặt vào! Chúng ta không thể để Thunder Beast gặp nguy hiểm!'}, {'character': 'Hùng', 'text': 'Tôi sẽ không để cơn bão này ngăn cản chúng ta!'}], 'final_transition': 'Sau khi vượt qua cơn bão, Thunder Beast và đội ngũ của nó tiếp tục hành trình với niềm tin mãnh liệt hơn bao giờ hết.'}, {'page': 3, 'title': 'Đến Nước Mỹ', 'scene_description': 'Thunder Beast cuối cùng cũng đặt chân đến Mỹ. Cả đội bị choáng ngợp bởi sự rộng lớn và nhộn nhịp của thành phố New York.', 'text_full': 'Khi Thunder Beast được đưa lên đất liền, cả đội không khỏi ngỡ ngàng trước sự hùng vĩ của thành phố New York. Những tòa nhà chọc trời, ánh đèn neon, và dòng người tấp nập khiến họ cảm thấy mình thật nhỏ bé.', 'image_prompt': 'comic style, highly detailed scene, dynamic perspective of a monster truck arriving in New York City, surrounded by skyscrapers and bustling streets, amazed team members', 'dialogue': [{'character': 'Nam', 'text': 'Chúng ta thực sự đã đến Mỹ rồi. Thunder Beast sẽ tỏa sáng ở đây!'}, {'character': 'Hùng', 'text': 'Đúng vậy, hãy cho họ thấy sức mạnh của chúng ta!'}], 'final_transition': 'Với tinh thần quyết tâm, cả đội chuẩn bị cho giải đấu sắp tới, nơi Thunder Beast sẽ chứng minh bản lĩnh của mình.'}, {'page': 4, 'title': 'Giải Đấu Đầu Tiên', 'scene_description': 'Thunder Beast tham gia giải đấu đầu tiên tại Mỹ. Đối thủ là những chiếc monster truck hùng mạnh, nhưng Thunder Beast không hề nao núng.', 'text_full': 'Trong sân vận động đông nghịt người, Thunder Beast bước vào trận đấu đầu tiên. Tiếng động cơ gầm rú, bụi bay mù mịt, và những pha biểu diễn ngoạn mục khiến khán giả không ngừng reo hò. Thunder Beast chứng tỏ mình là một đối thủ đáng gờm.', 'image_prompt': 'comic style, highly detailed scene, dynamic perspective of Thunder Beast competing in a monster truck rally in a packed stadium, dust flying, roaring engines, excited crowd', 'dialogue': [{'character': 'Nam', 'text': 'Hãy cho họ thấy sức mạnh của Thunder Beast!'}, {'character': 'Hùng', 'text': 'Đây là lúc chúng ta tỏa sáng!'}], 'final_transition': 'Với chiến thắng đầu tiên, Thunder Beast và đội ngũ của nó bắt đầu ghi dấu ấn tại nước Mỹ, mở ra một chương mới đầy hứa hẹn.'}]


[{'page': 1,
  'title': 'Khởi Đầu Cuộc Phiêu Lưu',
  'scene_description': "Một chiếc monster truck đầy màu sắc, tên là 'Thunder Beast', chuẩn bị rời Việt Nam để sang Mỹ tham gia một giải đấu lớn. Nhóm kỹ sư và tài xế đang kiểm tra lần cuối trước khi lên đường.",
  'text_full': 'Trong một xưởng sửa chữa nhộn nhịp tại Hà Nội, Thunder Beast, chiếc monster truck được trang bị động cơ mạnh mẽ và thiết kế độc đáo, đang được chuẩn bị cho chuyến đi định mệnh. Không khí hồi hộp và phấn khích bao trùm khi mọi người làm việc không ngừng nghỉ.',
  'image_prompt': 'comic style, highly detailed scene, dynamic perspective of a colorful monster truck being prepared in a bustling workshop, engineers and driver checking the vehicle, vibrant atmosphere',
  'dialogue': [{'character': 'Nam',
    'text': 'Mọi thứ đã sẵn sàng chưa? Chúng ta không thể để xảy ra bất kỳ sai sót nào!'},
   {'character': 'Hùng',
    'text': 'Yên tâm đi, Thunder Beast sẽ khiến cả nước Mỹ phải kinh ngạc!'}],
  'final_transition': '